# Survival analysis para la empresa JAMPP

El objetivo es analizar los datos de la empresa JAMPP para saber cuando un usuario volverá a conectarse a una app, dado sus datos de conexiones previas. La segunda predicción que intentaremos hacer es saber cuando volverá a "convertir", es decir comprar la app de la cual tiene una publicidad en su movil a un momento dado.



In [54]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import lifelines
import math
import category_encoders as ce
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
import xgboost

In [2]:
events = pd.read_csv('./events.csv')

/Users/yorickvinesse/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
auctions = pd.read_csv('./auctions.csv')

In [4]:
clicks = pd.read_csv('./clicks.csv')

/Users/yorickvinesse/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
installs = pd.read_csv('./installs.csv')

/Users/yorickvinesse/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
auctions.dtypes

date           object
device_id       int64
ref_type_id     int64
source_id       int64
dtype: object

In [4]:
auctions.count()

date           47409528
device_id      47409528
ref_type_id    47409528
source_id      47409528
dtype: int64

# Que datos nos conviene usar?

Acá estamos haciendo un _survival analysis_ entonces se necesita mostrar el momento al cual ocurrió un evento. En nuestro caso, si un evento aparece en el dataset de _auctions_, es porque ocurrió. El survival analysis se enfoca en la primera vez que ocurre un evento, puesto que hace una regresión de un set de caractéristicas en frente a una fecha de ocurrencia del evento.

Pero es cierto que muchos usuarios, aca representados por sus *device_id* aparecen varias veces en la lista de subastas (_auctions_), considerando que 475338 *device_id* únicos se pueden contar adentro del set de _auctions_ que almacena 47409528 observaciones distintas (100 veces más).

Dado eso, parecería inoportuno no considerar la característica de repetición de aparición de los usuarios cuando hacemos la predicción de su reaparición más adelante.

Podemos tratar de analizar datos como la cantidad de clicks hecha por un usuario antes de llevar a cabo una instalación, o la cantidad de veces que apareció en una subasta.

In [3]:
auctions.head(5)

,date,device_id,ref_type_id,source_id
0,2019-04-23 18:58:00.842116,2564673204772915246,1,0
1,2019-04-23 18:58:01.530771,4441121667607578179,7,0
2,2019-04-23 18:58:01.767562,7721769811471055264,1,0
3,2019-04-23 18:58:02.363468,6416039086842158968,1,0
4,2019-04-23 18:58:02.397559,1258642015983312729,1,0


In [23]:
auctions.date.max()

'2019-04-26 23:59:59.969518'

In [112]:
len(auctions.device_id.unique())

475338

# Cuandó aparecerá un usuario dado?

### Desarrollo alrededor del dataset _auctions_

Vamos a enriquecer el dataset relacionado a las subastas incluyendo, para cada observación, la cantidad de veces que ocurrió una aparición del mismo *device_id* dentro de la ventana de estudio anterior. También vamos a indicar el tiempo que pasó desde la última vez que ocurrió el evento para el mismo *device_id*.

Primero, queremos tener un modelo que nos indica cuando será la próxima vez que aparecerá un usuario en una subasta.

Es necesario convertir los tiempos de ocurrencia en segundos para que nos de una duración hasta el evento.

## Selección de las ventanas de tests de tres días en _Auctions_

In [3]:
auctionsVentanaUno = auctions.loc[(auctions["date"] < "2019-04-21")]
auctionsVentanaUno = auctionsVentanaUno.loc[(auctions["date"] > "2019-04-18")]

In [4]:
auctionsVentanaDos = auctions.loc[(auctions["date"] < "2019-04-22")]
auctionsVentanaDos = auctionsVentanaDos.loc[(auctions["date"] > "2019-04-19")]

In [5]:
auctionsVentanaTres = auctions.loc[(auctions["date"] < "2019-04-23")]
auctionsVentanaTres = auctionsVentanaTres.loc[(auctions["date"] > "2019-04-20")]

In [6]:
auctionsVentanaCuatro = auctions.loc[(auctions["date"] < "2019-04-24")]
auctionsVentanaCuatro = auctionsVentanaCuatro.loc[(auctions["date"] > "2019-04-21")]

In [7]:
auctionsVentanaCinco = auctions.loc[(auctions["date"] < "2019-04-25")]
auctionsVentanaCinco = auctionsVentanaCinco.loc[(auctions["date"] > "2019-04-22")]

In [8]:
auctionsVentanaSeis = auctions.loc[(auctions["date"] < "2019-04-26")]
auctionsVentanaSeis = auctionsVentanaSeis.loc[(auctions["date"] > "2019-04-23")]

In [9]:
auctionsVentanaSiete = auctions.loc[(auctions["date"] < "2019-04-27")]
auctionsVentanaSiete = auctionsVentanaSiete.loc[(auctions["date"] > "2019-04-24")]

### En todas las ventanas elegidas, vamos a agregar una columna de fecha inicial para poder basar los tiempos en base a eso. Despues podremos restar el tiempo inicial al tiempo absoluto para hacer modelos en segundos pasados.

In [10]:
auctionsVentanaUno['fechaInicial'] = pd.to_datetime("2019-04-18 00:00:00")
auctionsVentanaDos['fechaInicial'] = pd.to_datetime("2019-04-19 00:00:00")
auctionsVentanaTres['fechaInicial'] = pd.to_datetime("2019-04-20 00:00:00")
auctionsVentanaCuatro['fechaInicial'] = pd.to_datetime("2019-04-21 00:00:00")
auctionsVentanaCinco['fechaInicial'] = pd.to_datetime("2019-04-22 00:00:00")
auctionsVentanaSeis['fechaInicial'] = pd.to_datetime("2019-04-23 00:00:00")
auctionsVentanaSiete['fechaInicial'] = pd.to_datetime("2019-04-24 00:00:00")

In [15]:
auctionsVentanaUno.head()

,date,device_id,ref_type_id,source_id,fechaInicial
221726,2019-04-20 23:57:27.912838,1109595589636746168,7,0,2019-04-18
221727,2019-04-20 23:57:28.381114,5896614299191635403,1,0,2019-04-18
221728,2019-04-20 23:57:28.515423,4172466725848941608,1,0,2019-04-18
221729,2019-04-20 23:57:28.700884,2616279795187318849,7,0,2019-04-18
221730,2019-04-20 23:57:28.868312,8034952072073026056,1,0,2019-04-18


### Convertimos los objects de _date_ a elementos de tipo _datetime_ para poder calcular diferencias de tiempo y poder utilisar modelos de Lifelines.

In [11]:
auctionsVentanaUno.loc[:,'date'] = pd.to_datetime(auctionsVentanaUno['date'], errors = 'coerce')
auctionsVentanaDos.loc[:,'date'] = pd.to_datetime(auctionsVentanaDos['date'], errors = 'coerce')
auctionsVentanaTres.loc[:,'date'] = pd.to_datetime(auctionsVentanaTres['date'], errors = 'coerce')
auctionsVentanaCuatro.loc[:,'date'] = pd.to_datetime(auctionsVentanaCuatro['date'], errors = 'coerce')
auctionsVentanaCinco.loc[:,'date'] = pd.to_datetime(auctionsVentanaCinco['date'], errors = 'coerce')
auctionsVentanaSeis.loc[:,'date'] = pd.to_datetime(auctionsVentanaSeis['date'], errors = 'coerce')
auctionsVentanaSiete.loc[:,'date'] = pd.to_datetime(auctionsVentanaSiete['date'], errors = 'coerce')

### Sacamos la diferencia entre el tiempo de la ventana y el tiempo inicial de la ventana

In [12]:
auctionsVentanaUno["date"] = auctionsVentanaUno["date"] - auctionsVentanaUno["fechaInicial"]
auctionsVentanaDos["date"] = auctionsVentanaDos["date"] - auctionsVentanaDos["fechaInicial"]
auctionsVentanaTres["date"] = auctionsVentanaTres["date"] - auctionsVentanaTres["fechaInicial"]
auctionsVentanaCuatro["date"] = auctionsVentanaCuatro["date"] - auctionsVentanaCuatro["fechaInicial"]
auctionsVentanaCinco["date"] = auctionsVentanaCinco["date"] - auctionsVentanaCinco["fechaInicial"]
auctionsVentanaSeis["date"] = auctionsVentanaSeis["date"] - auctionsVentanaSeis["fechaInicial"]
auctionsVentanaSiete["date"] = auctionsVentanaSiete["date"] - auctionsVentanaSiete["fechaInicial"]

In [13]:
auctionsVentanaUno.head(2)

,date,device_id,ref_type_id,source_id,fechaInicial
221726,2 days 23:57:27.912838,1109595589636746168,7,0,2019-04-18
221727,2 days 23:57:28.381114,5896614299191635403,1,0,2019-04-18


In [113]:
auctionsVentanaUno.dtypes

date                        float64
device_id                     int64
ref_type_id                   int64
source_id                     int64
fechaInicial         datetime64[ns]
sum                           int64
cantidad_subastas             int64
dtype: object

### Queremos resultados en segundos

In [14]:
auctionsVentanaUno["date"] = auctionsVentanaUno["date"].dt.total_seconds()
auctionsVentanaDos["date"] = auctionsVentanaDos["date"].dt.total_seconds()
auctionsVentanaTres["date"] = auctionsVentanaTres["date"].dt.total_seconds()
auctionsVentanaCuatro["date"] = auctionsVentanaCuatro["date"].dt.total_seconds()
auctionsVentanaCinco["date"] = auctionsVentanaCinco["date"].dt.total_seconds()
auctionsVentanaSeis["date"] = auctionsVentanaSeis["date"].dt.total_seconds()
auctionsVentanaSiete["date"] = auctionsVentanaSiete["date"].dt.total_seconds()

Agregamos una columna _sum_ para poder contar la cantidad de veces que aparece un device en una ventana de tres días.

In [15]:
auctionsVentanaUno["sum"] = 1
auctionsVentanaDos["sum"] = 1
auctionsVentanaTres["sum"] = 1
auctionsVentanaCuatro["sum"] = 1
auctionsVentanaCinco["sum"] = 1
auctionsVentanaSeis["sum"] = 1
auctionsVentanaSiete["sum"] = 1

Contamos la cantidad de auctions en la ventana de 3 días para incluirla en la ventana siguiente como parametro.

In [16]:
cantidadDeSubastasPorIdVentanaUno = auctionsVentanaUno[['device_id','sum']].groupby('device_id').sum()
cantidadDeSubastasPorIdVentanaDos = auctionsVentanaDos[['device_id','sum']].groupby('device_id').sum()
cantidadDeSubastasPorIdVentanaTres = auctionsVentanaTres[['device_id','sum']].groupby('device_id').sum()
cantidadDeSubastasPorIdVentanaCuatro = auctionsVentanaCuatro[['device_id','sum']].groupby('device_id').sum()
cantidadDeSubastasPorIdVentanaCinco = auctionsVentanaCinco[['device_id','sum']].groupby('device_id').sum()
cantidadDeSubastasPorIdVentanaSeis = auctionsVentanaSeis[['device_id','sum']].groupby('device_id').sum()
cantidadDeSubastasPorIdVentanaSiete = auctionsVentanaSiete[['device_id','sum']].groupby('device_id').sum()

In [38]:
cantidadDeSubastasPorIdVentanaSeis.head(2)

,sum
device_id,
40621409780134,1
69039685746313,4


In [17]:
cantidadDeSubastasPorIdVentanaUno = cantidadDeSubastasPorIdVentanaUno.stack().reset_index()
cantidadDeSubastasPorIdVentanaDos = cantidadDeSubastasPorIdVentanaDos.stack().reset_index()
cantidadDeSubastasPorIdVentanaTres = cantidadDeSubastasPorIdVentanaTres.stack().reset_index()
cantidadDeSubastasPorIdVentanaCuatro = cantidadDeSubastasPorIdVentanaCuatro.stack().reset_index()
cantidadDeSubastasPorIdVentanaCinco = cantidadDeSubastasPorIdVentanaCinco.stack().reset_index()
cantidadDeSubastasPorIdVentanaSeis = cantidadDeSubastasPorIdVentanaSeis.stack().reset_index()
cantidadDeSubastasPorIdVentanaSiete = cantidadDeSubastasPorIdVentanaSiete.stack().reset_index()

In [18]:
cantidadDeSubastasPorIdVentanaUno = cantidadDeSubastasPorIdVentanaUno.drop(columns = ['level_1'])
cantidadDeSubastasPorIdVentanaDos = cantidadDeSubastasPorIdVentanaDos.drop(columns = ['level_1'])
cantidadDeSubastasPorIdVentanaTres = cantidadDeSubastasPorIdVentanaTres.drop(columns = ['level_1'])
cantidadDeSubastasPorIdVentanaCuatro = cantidadDeSubastasPorIdVentanaCuatro.drop(columns = ['level_1'])
cantidadDeSubastasPorIdVentanaCinco = cantidadDeSubastasPorIdVentanaCinco.drop(columns = ['level_1'])
cantidadDeSubastasPorIdVentanaSeis = cantidadDeSubastasPorIdVentanaSeis.drop(columns = ['level_1'])
cantidadDeSubastasPorIdVentanaSiete = cantidadDeSubastasPorIdVentanaSiete.drop(columns = ['level_1'])

In [19]:
cantidadDeSubastasPorIdVentanaUno.columns = ['device_id', 'cantidad_subastas']
cantidadDeSubastasPorIdVentanaDos.columns = ['device_id', 'cantidad_subastas']
cantidadDeSubastasPorIdVentanaTres.columns = ['device_id', 'cantidad_subastas']
cantidadDeSubastasPorIdVentanaCuatro.columns = ['device_id', 'cantidad_subastas']
cantidadDeSubastasPorIdVentanaCinco.columns = ['device_id', 'cantidad_subastas']
cantidadDeSubastasPorIdVentanaSeis.columns = ['device_id', 'cantidad_subastas']
cantidadDeSubastasPorIdVentanaSiete.columns = ['device_id', 'cantidad_subastas']

In [20]:
cantidadDeSubastasPorIdVentanaUno.head(2)

,device_id,cantidad_subastas
0,41863526108385,35
1,135153013040192,8


In [21]:
auctionsVentanaUno = pd.merge(auctionsVentanaUno, cantidadDeSubastasPorIdVentanaUno, on ='device_id', how = 'left')
auctionsVentanaDos = pd.merge(auctionsVentanaDos, cantidadDeSubastasPorIdVentanaDos, on ='device_id', how = 'left')
auctionsVentanaTres = pd.merge(auctionsVentanaTres, cantidadDeSubastasPorIdVentanaTres, on ='device_id', how = 'left')
auctionsVentanaCuatro = pd.merge(auctionsVentanaCuatro, cantidadDeSubastasPorIdVentanaCuatro, on ='device_id', how = 'left')
auctionsVentanaCinco = pd.merge(auctionsVentanaCinco, cantidadDeSubastasPorIdVentanaCinco, on ='device_id', how = 'left')
auctionsVentanaSeis = pd.merge(auctionsVentanaSeis, cantidadDeSubastasPorIdVentanaSeis, on ='device_id', how = 'left')
auctionsVentanaSiete = pd.merge(auctionsVentanaSiete, cantidadDeSubastasPorIdVentanaSiete, on ='device_id', how = 'left')


In [22]:
auctionsVentanaUno.head(7)

,date,device_id,ref_type_id,source_id,fechaInicial,sum,cantidad_subastas
0,259047.912838,1109595589636746168,7,0,2019-04-18,1,128
1,259048.381114,5896614299191635403,1,0,2019-04-18,1,46
2,259048.515423,4172466725848941608,1,0,2019-04-18,1,252
3,259048.700884,2616279795187318849,7,0,2019-04-18,1,191
4,259048.868312,8034952072073026056,1,0,2019-04-18,1,166
5,259049.202778,8018738201130876812,1,0,2019-04-18,1,1647
6,259049.441246,4914962826064649332,7,0,2019-04-18,1,58


### Despues haremos un feature con el tiempo promedio entre dos subastas por device_id.

### Para hacer eso, ordenaremos por device_id y despues por fecha, para poder correr una funcion que resta la diferencia entre dos elementos si tienen el mismo id

In [23]:
auctionsVentanaUno = auctionsVentanaUno.sort_values(['device_id', 'date'], ascending = [True, True])
auctionsVentanaDos = auctionsVentanaDos.sort_values(['device_id', 'date'], ascending = [True, True])
auctionsVentanaTres = auctionsVentanaTres.sort_values(['device_id', 'date'], ascending = [True, True])
auctionsVentanaCuatro = auctionsVentanaCuatro.sort_values(['device_id', 'date'], ascending = [True, True])
auctionsVentanaCinco = auctionsVentanaCinco.sort_values(['device_id', 'date'], ascending = [True, True])
auctionsVentanaSeis = auctionsVentanaSeis.sort_values(['device_id', 'date'], ascending = [True, True])
auctionsVentanaSiete = auctionsVentanaSiete.sort_values(['device_id', 'date'], ascending = [True, True])

In [24]:
auctionsVentanaUno.head(10)

,date,device_id,ref_type_id,source_id,fechaInicial,sum,cantidad_subastas
11212121,157228.465866,41863526108385,1,8,2019-04-18,1,35
5304186,183146.892880,41863526108385,1,3,2019-04-18,1,35
11604212,183542.509230,41863526108385,1,3,2019-04-18,1,35
14997595,183961.675788,41863526108385,1,3,2019-04-18,1,35
13857860,184137.388160,41863526108385,1,3,2019-04-18,1,35
8143403,184286.463903,41863526108385,1,3,2019-04-18,1,35
4680108,184301.011671,41863526108385,1,3,2019-04-18,1,35
13858166,184346.681062,41863526108385,1,5,2019-04-18,1,35
4680208,184377.482482,41863526108385,1,3,2019-04-18,1,35
13128758,184496.144108,41863526108385,1,3,2019-04-18,1,35


### Nos separamos de los duplicates para hacer una regresion por device. Nos quedaremos con ese set sin tocarlo para poder despues concatenar nuestros resultados sin device_id pero con el mismo orden. El orden nos indicara cual device_id es.

In [24]:
auctionsVentanaUnoRegressionSet = auctionsVentanaUno.drop_duplicates(subset = 'device_id')
auctionsVentanaDosRegressionSet = auctionsVentanaDos.drop_duplicates(subset = 'device_id')
auctionsVentanaTresRegressionSet = auctionsVentanaTres.drop_duplicates(subset = 'device_id')
auctionsVentanaCuatroRegressionSet = auctionsVentanaCuatro.drop_duplicates(subset = 'device_id')
auctionsVentanaCincoRegressionSet = auctionsVentanaCinco.drop_duplicates(subset = 'device_id')
auctionsVentanaSeisRegressionSet = auctionsVentanaSeis.drop_duplicates(subset = 'device_id')
auctionsVentanaSieteRegressionSet = auctionsVentanaSiete.drop_duplicates(subset = 'device_id')

In [25]:
auctionsVentanaUnoRegressionSet.head(5)

,date,device_id,ref_type_id,source_id,fechaInicial,sum,cantidad_subastas
11212121,157228.465866,41863526108385,1,8,2019-04-18,1,35
12832576,187854.009137,135153013040192,1,1,2019-04-18,1,8
15276817,10366.357746,161514654074162,1,1,2019-04-18,1,6
3972519,256765.420614,181891380775191,1,1,2019-04-18,1,1
5728752,60166.331894,186034136943920,1,1,2019-04-18,1,7


In [26]:
auctionsVentanaUnoRegressionSet = auctionsVentanaUnoRegressionSet[['cantidad_subastas', 'date', 'device_id']]
auctionsVentanaDosRegressionSet = auctionsVentanaDosRegressionSet[['cantidad_subastas', 'date', 'device_id']]
auctionsVentanaTresRegressionSet = auctionsVentanaTresRegressionSet[['cantidad_subastas', 'date', 'device_id']]
auctionsVentanaCuatroRegressionSet = auctionsVentanaCuatroRegressionSet[['cantidad_subastas', 'date', 'device_id']]
auctionsVentanaCincoRegressionSet = auctionsVentanaCincoRegressionSet[['cantidad_subastas', 'date', 'device_id']]
auctionsVentanaSeisRegressionSet = auctionsVentanaSeisRegressionSet[['cantidad_subastas', 'date', 'device_id']]
auctionsVentanaSieteRegressionSet = auctionsVentanaSieteRegressionSet[['cantidad_subastas', 'date', 'device_id']]

In [27]:
auctionsVentanaUnoRegressionSet = auctionsVentanaUnoRegressionSet.reset_index(drop = True)
auctionsVentanaDosRegressionSet = auctionsVentanaDosRegressionSet.reset_index(drop = True)
auctionsVentanaTresRegressionSet = auctionsVentanaTresRegressionSet.reset_index(drop = True)
auctionsVentanaCuatroRegressionSet = auctionsVentanaCuatroRegressionSet.reset_index(drop = True)
auctionsVentanaCincoRegressionSet = auctionsVentanaCincoRegressionSet.reset_index(drop = True)
auctionsVentanaSeisRegressionSet = auctionsVentanaSeisRegressionSet.reset_index(drop = True)
auctionsVentanaSieteRegressionSet = auctionsVentanaSieteRegressionSet.reset_index(drop = True)

In [54]:
auctionsVentanaUnoRegressionSet.head(2)

,cantidad_subastas,date,device_id
0,35,157228.465866,41863526108385
1,8,187854.009137,135153013040192


In [28]:
auctionsVentanaUnoRegressionSet.columns = ['cantidad_subastas_v1', 'date_v1', 'device_id']
auctionsVentanaDosRegressionSet.columns = ['cantidad_subastas_v2', 'date_v2', 'device_id']
auctionsVentanaTresRegressionSet.columns = ['cantidad_subastas_v3', 'date_v3', 'device_id']
auctionsVentanaCuatroRegressionSet.columns = ['cantidad_subastas_v4', 'date_v4', 'device_id']
auctionsVentanaCincoRegressionSet.columns = ['cantidad_subastas_v5', 'date_v5', 'device_id']
auctionsVentanaSeisRegressionSet.columns = ['cantidad_subastas_v6', 'date_v6', 'device_id']
auctionsVentanaSieteRegressionSet.columns = ['cantidad_subastas_v7', 'date_v7', 'device_id']

In [29]:
auctionsVentanaSieteRegressionSet.head(2)

,cantidad_subastas_v7,date_v7,device_id
0,4,126258.597103,69039685746313
1,2,250362.048531,345999128501141


In [30]:
auctionsV_1_2_3 = pd.merge(auctionsVentanaUnoRegressionSet, auctionsVentanaDosRegressionSet,\
                           on = "device_id", how = "outer")

In [31]:
auctionsV_1_2_3 = pd.merge(auctionsV_1_2_3, auctionsVentanaTresRegressionSet,\
                           on = "device_id", how = "outer")

### Tomamos tambien el set sobre lo cual aplicaremos el fit despues para predecir el target, aca auctionsV_5_6_7

In [71]:
auctionsV_5_6_7 = pd.merge(auctionsVentanaCincoRegressionSet, auctionsVentanaSeisRegressionSet,\
                           on = "device_id", how = "outer")

In [72]:
auctionsV_5_6_7 = pd.merge(auctionsV_5_6_7, auctionsVentanaSieteRegressionSet,\
                           on = "device_id", how = "outer")

In [32]:
auctionsV_1_2_3.head(2)

,cantidad_subastas_v1,date_v1,device_id,cantidad_subastas_v2,date_v2,cantidad_subastas_v3,date_v3
0,35.0,157228.465866,41863526108385,35.0,70828.465866,34.0,10346.892880
1,8.0,187854.009137,135153013040192,8.0,101454.009137,8.0,15054.009137


In [33]:
targetV_6 = auctionsVentanaSeisRegressionSet[['date_v6', 'device_id']]

In [34]:
targetV_6.head(2)

,date_v6,device_id
0,54057.222979,40621409780134
1,212658.597103,69039685746313


In [35]:
trainingSetV_1_2_3_6 = pd.merge(auctionsV_1_2_3, targetV_6, on = "device_id", how = "left")

In [36]:
trainingSetV_1_2_3_6[['cantidad_subastas_v1', 'cantidad_subastas_v2', 'cantidad_subastas_v3']] = \
trainingSetV_1_2_3_6[['cantidad_subastas_v1', 'cantidad_subastas_v2', 'cantidad_subastas_v3']].fillna(value = 0)

In [38]:
trainingSetV_1_2_3_6[['date_v1', 'date_v2', 'date_v3', 'date_v6']] = \
trainingSetV_1_2_3_6[['date_v1', 'date_v2', 'date_v3', 'date_v6']].fillna(value = 259200)

### Eso que sigue para predecir el target

In [74]:
auctionsV_5_6_7[['cantidad_subastas_v5', 'cantidad_subastas_v6', 'cantidad_subastas_v7']] = \
auctionsV_5_6_7[['cantidad_subastas_v5', 'cantidad_subastas_v6', 'cantidad_subastas_v7']].fillna(value = 0)

In [75]:
auctionsV_5_6_7[['date_v5', 'date_v6', 'date_v7']] = \
auctionsV_5_6_7[['date_v5', 'date_v6', 'date_v7']].fillna(value = 259200)

In [39]:
trainingSetV_1_2_3_6.head(2)

,cantidad_subastas_v1,date_v1,device_id,cantidad_subastas_v2,date_v2,cantidad_subastas_v3,date_v3,date_v6
0,35.0,157228.465866,41863526108385,35.0,70828.465866,34.0,10346.892880,259200.0
1,8.0,187854.009137,135153013040192,8.0,101454.009137,8.0,15054.009137,259200.0


In [76]:
auctionsV_5_6_7.head(2)

,cantidad_subastas_v5,date_v5,device_id,cantidad_subastas_v6,date_v6,cantidad_subastas_v7,date_v7
0,1.0,140457.222979,40621409780134,1.0,54057.222979,0.0,259200.0
1,1.0,86644.526198,168103949904656,1.0,244.526198,0.0,259200.0


In [40]:
trainingSetV_1_2_3_6sinId = trainingSetV_1_2_3_6.drop(columns = ['device_id'])

In [78]:
auctionsV_5_6_7sinId = auctionsV_5_6_7.drop(columns = ['device_id'])

### Le damos una forma NumPy Array para ingresarlo a los fitters.

In [41]:
arraytrainingSetV_1_2_3_6 = trainingSetV_1_2_3_6sinId.values

### Misma forma para el que necesitamos para predecir el target

In [80]:
X_arrayauctionsV_5_6_7sinId = auctionsV_5_6_7sinId.values

In [42]:
len(arraytrainingSetV_1_2_3_6)

389603

In [43]:
X_arraytrainingSetV_1_2_3_6 = arraytrainingSetV_1_2_3_6[:, :-1]
y_arraytrainingSetV_1_2_3_6 = arraytrainingSetV_1_2_3_6[:, -1]

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X_arraytrainingSetV_1_2_3_6,\
                                                    y_arraytrainingSetV_1_2_3_6, test_size=0.2, random_state=42)

### Con regresion

In [45]:
regTrainingSetV_1_2_3_6 = LinearRegression().fit(X_train, y_train)

In [46]:
y_estim = regTrainingSetV_1_2_3_6.predict(X_test)

In [47]:
rmseRegLin = math.sqrt(mean_squared_error(y_test, y_estim))

In [48]:
rmseRegLin

98372.71602602713

In [55]:
regTrainingSetV_1_2_3_6.score(X_train, y_train)

0.1459743415712349

### Avec XGBoost 

In [49]:
xgbV_1_2_3_6 = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7, objective='reg:squarederror')

In [50]:
xgbV_1_2_3_6.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.08, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=0.75, verbosity=1)

In [51]:
y_estim = xgbV_1_2_3_6.predict(X_test)

In [52]:
rmseXGB = math.sqrt(mean_squared_error(y_test, y_estim))

In [53]:
rmseXGB

91293.24236426195

### Con Ridge alpha = 100

In [66]:
RidgeV_1_2_3_6 = Ridge(alpha = 100)

In [67]:
RidgeV_1_2_3_6.fit(X_train, y_train)

Ridge(alpha=100, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [68]:
y_estim = RidgeV_1_2_3_6.predict(X_test)

In [69]:
rmseRidge1 = math.sqrt(mean_squared_error(y_test, y_estim))

In [70]:
rmseRidge1

98372.71603035436

### en random forest

In [108]:
rfV_1_2_3_7 = RandomForestRegressor(n_estimators = 1000, random_state = 42)

In [109]:
rfV_1_2_3_7.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [110]:
y_estim = rfV_1_2_3_7.predict(X_test)

In [111]:
rmse = math.sqrt(mean_squared_error(y_test, y_estim))

In [112]:
rmse

100770.00664961891

### Usamos el fitter de XGBoost para predecir el target

In [81]:
y_targetXGBoost = xgbV_1_2_3_6.predict(X_arrayauctionsV_5_6_7sinId)

In [82]:
dfPredictXGBoost = pd.DataFrame(y_targetXGBoost, columns = ['prediccion_subasta'])

In [83]:
df_device_auctionsV_5_6_7 = auctionsV_5_6_7[['device_id']]

In [85]:
df_device_auctionsV_5_6_7 = df_device_auctionsV_5_6_7.reset_index(drop = True)

In [87]:
dfPredictXGBoost = pd.concat([df_device_auctionsV_5_6_7, dfPredictXGBoost], axis = 1)

In [89]:
len(dfPredictXGBoost)

395239

In [90]:
dfPredictXGBoost['device_id'] = dfPredictXGBoost.device_id.astype(str) + '_st'

In [95]:
dfPredictXGBoost.columns = ['ref_hash', 'prediccion_subasta']

In [96]:
dfPredictXGBoost.head(2)

,ref_hash,prediccion_subasta
0,40621409780134_st,206127.28125
1,168103949904656_st,186466.50000


### Ahora a leer el archivo llenado con los Sc, para ponerle los valores St

In [91]:
targetConSc = pd.read_csv('./targetConSc.csv')

In [93]:
targetConSc.head()

,ref_hash,obj
0,1000169251625791246_sc,259200
1,1000169251625791246_st,0
2,1000395625957344683_sc,161908
3,1000395625957344683_st,0
4,1003027494996471685_sc,182100


In [97]:
targetConScConSt = pd.merge(targetConSc, dfPredictXGBoost, on = 'ref_hash', how = 'left')

In [99]:
targetConScConSt.head(10)

,ref_hash,obj,prediccion_subasta
0,1000169251625791246_sc,259200,NaN
1,1000169251625791246_st,0,73578.976562
2,1000395625957344683_sc,161908,NaN
3,1000395625957344683_st,0,46671.476562
4,1003027494996471685_sc,182100,NaN
5,1003027494996471685_st,0,118290.710938
6,1006670001679961544_sc,195579,NaN
7,1006670001679961544_st,0,150629.578125
8,1007573308966476713_sc,259200,NaN
9,1007573308966476713_st,0,95972.078125


In [100]:
targetConScConSt.loc[targetConScConSt.index % 2 == 0, 'prediccion_subasta'] = 0

In [103]:
targetConScConSt = targetConScConSt.fillna(value = 259200)

In [104]:
targetConScConSt.head(2)

,ref_hash,obj,prediccion_subasta
0,1000169251625791246_sc,259200,0.000000
1,1000169251625791246_st,0,73578.976562


In [105]:
targetConScConSt['prediccion_subasta'] = targetConScConSt.prediccion_subasta.round().astype(int)

In [107]:
targetConScConSt['obj'] = targetConScConSt['obj'] + targetConScConSt['prediccion_subasta']

In [108]:
targetConScConSt.head(2)

,ref_hash,obj,prediccion_subasta
0,1000169251625791246_sc,259200,0
1,1000169251625791246_st,73579,73579


In [109]:
targetConScConSt = targetConScConSt.drop(columns = ['prediccion_subasta'])

In [110]:
targetConScConSt.head()

,ref_hash,obj
0,1000169251625791246_sc,259200
1,1000169251625791246_st,73579
2,1000395625957344683_sc,161908
3,1000395625957344683_st,46671
4,1003027494996471685_sc,182100


In [111]:
targetConScConSt.to_csv('targetConScConSt.csv', encoding='utf-8', index=False)

In [325]:
targetUpdated.to_csv('targetConSc.csv', encoding='utf-8', index=False)